## Convolutional Neural Network (CNN) PyTorch版

### SetUp

**GPU**

### MNISTデータを用意

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])
trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [2]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [3]:
print(len(trainset))
data, label = trainset[0]
print(type(data))
print(type(label))
digit_data = data.numpy()
print(digit_data.shape)

60000
<class 'torch.Tensor'>
<class 'int'>
(1, 28, 28)


In [4]:
import numpy as np
from PIL import Image
from IPython.display import display
img = digit_data.reshape(28,28)
pil_img = Image.fromarray(np.uint8(img))
display(pil_img)

**main.py - model setup and training**

<img src="https://miro.medium.com/max/1000/1*4OUonEDfZwCfR4Y-G-h1fw.jpeg" width=700>

**Program Setup**

In [5]:
!git clone https://github.com/pytorch/examples.git

Cloning into 'examples'...
remote: Enumerating objects: 3484, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 3484 (delta 6), reused 33 (delta 1), pack-reused 3413
Receiving objects: 100% (3484/3484), 40.40 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (1690/1690), done.


In [6]:
%cd examples/mnist

/content/examples/mnist


In [7]:
!ls

main.py  README.md  requirements.txt


In [8]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install tqdm
import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**プログラム本体**

In [10]:
#Python2系との互換。無視
#from __future__ import print_function

#コマンドラインを解析し、Pythonデータ型に変換
import argparse

#torchライブラリーを呼び込む
import torch

#WHAT IS TORCH.NN REALLY?
#ref. https://pytorch.org/tutorials/beginner/nn_tutorial.html
import torch.nn as nn

#refactoring bare codes
import torch.nn.functional as F

#最適化パッケージ
import torch.optim as optim

# Pytorchが提供する画像データおよびPytorch用データ変換ライブラリ
from torchvision import datasets, transforms

#エポック単位での学習率変化を自動計算
#1epock=(データ60000/(バッチサイズ64*log-interval(=10))単位で損失計算
# ⇒全単位終了後の損失平均
from torch.optim.lr_scheduler import StepLR # 標準的なスケジューラー

#pytorchネットワークモジュール定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #入力チャネル=1(白黒仮定), 出力チャネル=32, カーネルサイズ=3, ストライド=1
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        #forward propagationでは確率p(=0.25)で入力が０となり、
        #それ以外は入力が1/(1-p)でスケールアップする。
        #back propagationでは勾配に対し確率pで０化され、それ以外は1/(1-p)でスケールアップする。
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        #入力特徴数=9216、出力特徴数=128 全結合ネットワーク
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        #２回にわたり特徴面を拡大。
        #InputShape=28*28*1 OutputShape=26*26*32, Weights=(3*3*1)*32=288
        x = self.conv1(x)
        #torchが提供するランプ関数 負部分の値をゼロに平滑化する
        x = F.relu(x)
        #InputShape=26*26*32, OutputShape=24*24*64, Weights=(3*3*32)*64=18342
        x = self.conv2(x)
        x = F.relu(x)
        #カーネルサイズ=2 (入力2*2範囲内の最大値を1セルで置き換える)
        #InputShape=24*24*64, OutputShape=12*12*64(=9216)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        #InputShape=12*12*64, OutputShape=9216
        x = torch.flatten(x, 1)
        #Weight=9216*128=1179648
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        #Weghts=128*10=1280
        x = self.fc2(x)
        #-- Total Weights=1199558~=1.2M parameters
        #解説
        #softmax: e(x_i)/e(x).sum()
        #log_softmax: log(softmax(x))
        #dim=1 row-wise
        output = F.log_softmax(x, dim=1)
        return output

def train(args, model, device, train_loader, optimizer, epoch):
    #1エポックごとに呼び出される。
    #modelを学習用にセットアップする。
    model.train()
    #データをbatch-size単位に切り出し、(data, target)に再代入する
    for batch_idx, (data, target) in enumerate(train_loader):
        #data, targetをdevice（たとえばGPU）に展開する。
        data, target = data.to(device), target.to(device)
        #ミニバッチ単位の勾配計算に関し、勾配値の集積を毎回リセットする。
        optimizer.zero_grad()
        #出力を得る
        output = model(data)
        #損失を計算。 a:softmax(x), nll: -log(a)
        loss = F.nll_loss(output, target)
        #loss = F.cross_entropy(output, target)
        #勾配値を計算し、全パラメータを更新する。
        loss.backward()
        #指定された最適化関数の最適化方向へ勾配をシフト
        optimizer.step()
        #batch-size(=64)*log-interval(=50)ごとに損失率を記録。
        #loss.item() 損失平均
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

def test(model, device, test_loader):
    #学習時と異なるネットワーク構成（たとえばdropoutを含む）の場合。
    model.eval()
    test_loss = 0
    correct = 0
    #model.train()で学習済みのため、テスト時、勾配計算を省く。これによって推論時速度が上がる。
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            #test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            #出力中、もっとも高い確率を持つインデックスを知る。
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            #正解数を計算する。
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=20, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=50, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    ##本来、pythonインラインで実行を想定したプログラムをjupyterで動かす際の引数渡しに
    ##関する変更
    #args = parser.parse_args(), replaced with the following code for run in Jupyter
    args = parser.parse_args(args=[])
    #CUDA、つまりGPU使用の可否を確認。
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    #use_cuda = False

    #全体で使用する乱数シードの設定。
    torch.manual_seed(args.seed)

    #使用するデバイスの決定。cpuもしくはgpu
    device = torch.device("cuda" if use_cuda else "cpu")

    #cuda用学習時パラメータの用意
    #num_workers: 並列バッチ数
    kwargs = {'batch_size': args.batch_size}
    if use_cuda:
        kwargs.update({'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True},
                     )
    #入力データ(ndarray)をTensor値に変更
    #image = (image - mean) / std, here mean=0.1307, std=0.3081
    transform=transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize((0.1307,), (0.3081,))
        transforms.Normalize((0.5,), (0.5,))
        ])
    #学習前データ
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    #学習済みデータ
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    #ミニバッチを前提としたデータ取り込み
    train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

    #netモデルをデバイスに展開
    model = Net().to(device)
    #最適化モデルの選択
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
    #optimizer = optim.Adam(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in tqdm.tqdm(range(1, args.epochs + 1)):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == '__main__':
    main()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0279, Accuracy: 9914/10000 (99%)

Train Epoch: 19 [0/60000 (0%)]	Loss: 0.059340
Train Epoch: 19 [3200/60000 (5%)]	Loss: 0.034882
Train Epoch: 19 [6400/60000 (11%)]	Loss: 0.009266
Train Epoch: 19 [9600/60000 (16%)]	Loss: 0.016207
Train Epoch: 19 [12800/60000 (21%)]	Loss: 0.007643
Train Epoch: 19 [16000/60000 (27%)]	Loss: 0.000291
Train Epoch: 19 [19200/60000 (32%)]	Loss: 0.014314
Train Epoch: 19 [22400/60000 (37%)]	Loss: 0.004982
Train Epoch: 19 [25600/60000 (43%)]	Loss: 0.038056
Train Epoch: 19 [28800/60000 (48%)]	Loss: 0.018157
Train Epoch: 19 [32000/60000 (53%)]	Loss: 0.010609
Train Epoch: 19 [35200/60000 (59%)]	Loss: 0.014799
Train Epoch: 19 [38400/60000 (64%)]	Loss: 0.007933
Train Epoch: 19 [41600/60000 (69%)]	Loss: 0.014819
Train Epoch: 19 [44800/60000 (75%)]	Loss: 0.017743
Train Epoch: 19 [48000/60000 (80%)]	Loss: 0.014771
Train Epoch: 19 [51200/60000 (85%)]	Loss: 0.120904
Train Epoch: 19 [54400/60000 (91%)]	Loss: 0.019149
Train Epoch: 19 [57600/60000 (

 95%|█████████▌| 19/20 [04:46<00:15, 15.03s/it]


Test set: Average loss: 0.0279, Accuracy: 9914/10000 (99%)

Train Epoch: 20 [0/60000 (0%)]	Loss: 0.024323
Train Epoch: 20 [3200/60000 (5%)]	Loss: 0.018321
Train Epoch: 20 [6400/60000 (11%)]	Loss: 0.082088
Train Epoch: 20 [9600/60000 (16%)]	Loss: 0.004700
Train Epoch: 20 [12800/60000 (21%)]	Loss: 0.008565
Train Epoch: 20 [16000/60000 (27%)]	Loss: 0.022052
Train Epoch: 20 [19200/60000 (32%)]	Loss: 0.036140
Train Epoch: 20 [22400/60000 (37%)]	Loss: 0.014923
Train Epoch: 20 [25600/60000 (43%)]	Loss: 0.120426
Train Epoch: 20 [28800/60000 (48%)]	Loss: 0.034195
Train Epoch: 20 [32000/60000 (53%)]	Loss: 0.001766
Train Epoch: 20 [35200/60000 (59%)]	Loss: 0.015588
Train Epoch: 20 [38400/60000 (64%)]	Loss: 0.020368
Train Epoch: 20 [41600/60000 (69%)]	Loss: 0.004962
Train Epoch: 20 [44800/60000 (75%)]	Loss: 0.010044
Train Epoch: 20 [48000/60000 (80%)]	Loss: 0.025231
Train Epoch: 20 [51200/60000 (85%)]	Loss: 0.009246
Train Epoch: 20 [54400/60000 (91%)]	Loss: 0.123385
Train Epoch: 20 [57600/60000 (

100%|██████████| 20/20 [05:01<00:00, 15.08s/it]


Test set: Average loss: 0.0280, Accuracy: 9914/10000 (99%)



## Kearas版

ref. https://keras.io/examples/vision/mnist_convnet/

**Convolutional Neural Network CNN**

<img src="https://miro.medium.com/max/1000/1*4OUonEDfZwCfR4Y-G-h1fw.jpeg" width=700>

The output of the Flattened Layer will be the input to MLP(Dense Layer). However the size of input in the MLP example is 16000 (784(=28*28) in the previous MLP case) and featurized with Convolutional Network.

Explainable MNIST classification: dissection of a ConvNet ( https://towardsdatascience.com/explainable-mnist-classification-dissection-of-a-convnet-f32910d52842 )

**conv1**
Let’s start by looking at the weights of conv1 which has shape [8, 1, 5, 5] (8 outer channels, 1 inner channel, 5x5 grids):
<img src="https://miro.medium.com/max/1400/1*Qb7A_ZGGkri5McsKm0qldw.png">

and their activations on random input from MNIST:
<img src="https://miro.medium.com/max/1400/1*IkdsnYnKwfNXtPfe5nwAyg.png">
and their activations on synthetic input (in the image, I used a line for each direction: horizontal, vertical, and 2 diagonals):

<img src="https://miro.medium.com/max/1400/1*smBVTDAIQ48YTvjn5IBtIg.png">
The 3 images above strongly suggest that the first filter in conv1 detects lines at slope=45° (you can see that just from the weights — the pattern resembles a line at slope=45°). 

**conv2** digit 0

<img src="https://miro.medium.com/max/1400/1*iHxroAWMZbHi9FYxj9uRgg.png">

**conv3** digit 5

<img src="https://miro.medium.com/max/1400/1*HXjq7ngeWsTc5pFO-IgWXw.png">



Setup

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

Prepare the data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)  #28*28 black&white

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train / 255
x_test = x_test / 255
print("x_train shape:", x_train.shape)
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

Build the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),    #the output of this layer will be the input to MLP
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"), #MLP
    ]
)

model.summary()

Train the model

In [ ]:
batch_size = 128
epochs = 15


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Evaluate the trained model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])